### 1d

In [1]:
import os
import sys
import inspect

from IPython.display import display

# insert hybriddomain dir into sys
# hybriddomain must contain envs folder:
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
hd = currentdir.find("envs")
hd_dir = currentdir[:hd]
print(hd_dir)

if hd_dir not in sys.path:
    sys.path.insert(0, hd_dir)

env_dir = os.path.join(hd_dir, 'math_space', "common")
print(env_dir)

if env_dir not in sys.path:
    sys.path.insert(0, env_dir)


/home/valdecar/Documents/projects/projectsNew/lab/hybriddomain/
/home/valdecar/Documents/projects/projectsNew/lab/hybriddomain/math_space/common


### Описание модели

Загрузка шаблона

In [2]:
from envs.hs.model.model_main import ModelNet as Model

m = Model()
m.io.loadFromFile('../../../problems/1dTests/test1d_two_blocks0')


INFO:block_main.block_io:DefaultBound not set
INFO:block_main.block_io:DefaultBound not set


In [3]:
m.project_path

'../../../1dTests/test1d_two_blocks0'

##### Блоки

Для одномерного случая у блока существует только сторона 2 (номер верхней стороны 2d блока). Она используется здесь только для хранения equationRegions, поскольку boundRegions для 1d являются границами отрезков т.е. точки.

In [18]:
display("block 0 side:")
display(m.blocks[0].sides[2].__repr__(notebook=True))
display("block 1 side:")
display(m.blocks[1].sides[2].__repr__(notebook=True))


'block 0 side:'

,0,1,2,3,4
bound/eq,"(None, 0)","(None, 2)","(None, 2)","(None, 2)","(None, 0)"
range,"[0.0, 0.5]","[0.5, 2.5]","[2.5, 2.7]","[2.7, 3.0]","[3.0, 9.0]"


'block 1 side:'

,0
bound/eq,"(None, 0)"
range,"[0.0, 1.0]"


##### Интерконнекты

In [4]:
m.interconnects

[
 ic: connection
 blockNumber1: 0
 blockNumber2: 1
 block1Side: 1
 block2Side: 0]

### Изменение модели

##### Создание блока. 

Регион e0 добавляется через сторону, e1 через блок. Для 1d сторона всегда 2.

In [5]:
from math_space.pde.regions import BoundRegion, EquationRegion
from envs.hs.block.side.side_main import SideNet as Side
from envs.hs.block.block_size import BlockSize
from envs.hs.block.block_main import BlockNet as Block

e0 = EquationRegion(dim=1, EquationNumber=1,
                    xfrom=0.1, xto=0.2)

e1 = EquationRegion(dim=1, EquationNumber=2,
                    xfrom=0.2, xto=0.3)

s = Side(2, eRegions=[e0], dim=1)
    
size = BlockSize()
size.set_default(dimension=1)
        
b = Block(name="Block 1", size=size, sides=[s],
          eRegions=[e1])

Теперь у блока оба региона в equationRegions.

In [6]:
for eRegion in b.equationRegions:
        display(eRegion)
# s.eRegions


   from   to
x   0.1  0.2
equationNumber: 1

   from   to
x   0.2  0.3
equationNumber: 2

##### Добавление правого граничного условия для блока.

В случае 1d block.boundRegions[0] это левая а block.boundRegions[1] правая граница блока, в то время как block.sides[2] представляет собой сам блок (точнее итервал). В отличае от этого в 2d все индексы bounRegions и sides совпадают.

boundNumber  это номер уравнения в model.bounds, а не индекс boundRegions

Вторая сторона будет использована (0) для интерконнекта

In [7]:
br = BoundRegion(BoundNumber=3, Side=1, dim=1)
b.editor.add_bound_region(br)
b.boundRegions

{0: [], 1: [
  side_num: 1 
  boundNumber: 3], 2: [], 3: []}

In [8]:
p = b.plotter.plot()
display(p)

,left vertex,1,2,3,4,right vertex
bound/eq,"(None, 0)","(None, 0)","(None, 1)","(None, 2)","(None, 0)","(3, 0)"
range,0,"[0.0, 0.1]","[0.1, 0.2]","[0.2, 0.3]","[0.3, 1.0]",sizeX


Одновременно с bound регионом добаляются вершины. В данном случае (3,0) означает model.bounds[3]

##### Добавление нового equationRegion:

In [9]:
e2 = EquationRegion(dim=1, EquationNumber=2,
                    xfrom=0.5, xto=1.0)
b.editor.add_eq_region(e2)
b.equationRegions

[   from   to
 x   0.1  0.2
 equationNumber: 1,    from   to
 x   0.2  0.3
 equationNumber: 2,    from   to
 x   0.5  1.0
 equationNumber: 2]

In [10]:
p = b.plotter.plot()
display(p)

,left vertex,1,2,3,4,5,right vertex
bound/eq,"(None, 0)","(None, 0)","(None, 1)","(None, 2)","(None, 0)","(None, 2)","(3, 2)"
range,0,"[0.0, 0.1]","[0.1, 0.2]","[0.2, 0.3]","[0.3, 0.5]","[0.5, 1.0]",sizeX


Одновременно с equation регионом добаляются вершины. В данном случае (3,2) означает model.equations[2]

##### Добавление блока

In [11]:
m.editor.add_block(b)

##### Добавление интерконнекта

Создание

In [12]:
from envs.hs.interconnect.ic_main import icMain as Interconnect

ic = Interconnect("3", model=m,
                  blockNumber1=1, blockNumber2=2,
                  block1Side=0, block2Side=0)

Добавление

In [13]:
m.editor.add_ic(ic)

obj alredy exist:

ic: 3
blockNumber1: 1
blockNumber2: 2
block1Side: 0
block2Side: 0



In [14]:
p = ic.plotter.plot()
for pp in p:
    display(pp)

,left vertex,1,right vertex
bound/eq,"(None, 0)","(None, 0)","(0, 0)"
range,0,"[0.0, 1.0]",sizeX


,left vertex,1,2,3,4,5,right vertex
bound/eq,"(None, 0)","(None, 0)","(None, 1)","(None, 2)","(None, 0)","(None, 2)","(3, 2)"
range,0,"[0.0, 0.1]","[0.1, 0.2]","[0.2, 0.3]","[0.3, 0.5]","[0.5, 1.0]",sizeX


In [15]:
m.blocks[2].boundRegions

{0: [], 1: [
  side_num: 1 
  boundNumber: 3], 2: [], 3: []}

In [16]:
m.blocks[2].equationRegions

[   from   to
 x   0.1  0.2
 equationNumber: 1,    from   to
 x   0.2  0.3
 equationNumber: 2,    from   to
 x   0.5  1.0
 equationNumber: 2]